Nombre:  Wilson Inga

# Ejercicio 2
### ¿Cuántos términos exiten en el siguiente corpus?

In [28]:
def contar_terminos(documento):
    total_terminos = 0
    with open(documento, 'r', encoding='utf-8') as file:
        for linea in file:
            terminos = linea.split()
            total_terminos += len(terminos)

    print(f"\nTotal de términos en el documento: {total_terminos}")

In [29]:
contar_terminos('c:/Users/wil_s/Documentos/GitHub/2025A/ir25a/data/01_corpus_turismo.txt')


Total de términos en el documento: 240


### Normalización de Términos

In [30]:
import unicodedata, string, re
from collections import Counter

def normalizar(texto):
    texto = unicodedata.normalize('NFD', texto)
    texto = ''.join(c for c in texto if unicodedata.category(c) != 'Mn')
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    return texto.lower()

def contar_terminos(ruta):
    with open(ruta, 'r', encoding='utf-8') as f:
        palabras = re.findall(r'\b\w+\b', normalizar(f.read()))
    frec = Counter(palabras)
    print(f"\nTotal de términos únicos: {len(frec)}")
    for t in sorted(frec): print(f"{t}: {frec[t]}")

In [31]:
contar_terminos('c:/Users/wil_s/Documentos/GitHub/2025A/ir25a/data/01_corpus_turismo.txt')


Total de términos únicos: 145
2000: 1
a: 6
agua: 1
alrededor: 1
altitud: 1
amazonia: 1
andes: 1
arquitectura: 1
artesanal: 1
ascensiones: 1
atrae: 1
atraen: 1
aventura: 1
aves: 1
avistamiento: 1
banos: 1
bienestar: 1
biodiversidad: 1
bosques: 1
cada: 1
cajas: 1
caminatas: 1
canopy: 1
cascadas: 1
centro: 1
cercania: 1
ciudad: 1
colonial: 1
color: 1
como: 3
comunidades: 1
con: 5
conecta: 1
conocer: 1
conocido: 1
cotopaxi: 1
crater: 1
cuenca: 1
cuenta: 1
cultura: 1
de: 16
declarado: 1
del: 3
deportes: 1
deslumbra: 1
disfrutan: 2
durante: 1
ecologico: 1
ecuador: 1
ecuatorianos: 1
el: 9
en: 4
entre: 1
es: 3
excursionistas: 1
exploracion: 1
famosas: 1
famoso: 1
feriado: 2
feriados: 1
fiestas: 1
galapagos: 1
gastronomia: 2
gran: 1
guayaquil: 1
historico: 1
humanidad: 1
ideales: 1
impacta: 1
impresionantes: 1
indigena: 1
indigenas: 1
interesados: 1
invita: 1
islas: 1
jovenes: 1
la: 10
laguna: 1
lagunas: 1
las: 4
llena: 1
locales: 1
longevidad: 1
lopez: 1
los: 7
malecon: 1
mas: 1
megadiverso: 

### Cálculo DF (Número de documentos donde aparece el término) e IDF

In [32]:
import math, pandas as pd

def normalizar(texto):
    texto = unicodedata.normalize('NFD', texto)
    texto = ''.join(c for c in texto if unicodedata.category(c) != 'Mn')
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    return texto.lower()

def matriz_idf(documento):
    with open(documento, 'r', encoding='utf-8') as f:
        lineas = [set(normalizar(p) for p in l.split() if normalizar(p)) for l in f]
    terminos = sorted(set.union(*lineas))
    total = len(lineas)
    matriz = []
    for t in terminos:
        fila = [int(t in l) for l in lineas]
        df = sum(fila)
        idf = round(math.log10(total / df), 4) if df else 0
        matriz.append(fila + [df, idf])
    cols = [f'doc{i+1}' for i in range(total)] + ['DF', 'IDF']
    return pd.DataFrame(matriz, index=terminos, columns=cols).rename_axis("Término")


In [42]:
df = matriz_idf('c:/Users/wil_s/Documentos/GitHub/2025A/ir25a/data/01_corpus_turismo.txt')
pd.set_option('display.max_columns', None)
pd.set_option("display.width", None)
pd.set_option('display.max_rows', None)
print(df)

                doc1  doc2  doc3  doc4  doc5  doc6  doc7  doc8  doc9  doc10  \
Término                                                                       
2000               0     0     0     0     0     0     0     0     0      1   
a                  0     0     0     0     0     0     1     1     0      1   
agua               0     0     0     1     0     0     0     0     0      0   
alrededor          0     0     0     0     0     0     0     0     0      0   
altitud            0     0     0     0     0     0     0     0     0      0   
amazonia           0     0     0     0     0     0     0     0     0      0   
andes              0     0     0     0     0     0     0     1     0      0   
arquitectura       0     0     0     0     0     0     1     0     0      0   
artesanal          0     0     0     0     0     1     0     0     0      0   
ascensiones        0     0     0     0     0     0     0     1     0      0   
atrae              0     0     0     0     0     0  

### Matriz TF-IDF

In [38]:
import math, pandas as pd

def matriz_tfidf(documento):
    with open(documento, 'r', encoding='utf-8') as f:
        documentos = [normalizar(l).split() for l in f if l.strip()]
    
    total_docs = len(documentos)
    terminos = sorted(set(p for doc in documentos for p in doc))
    
    # Calcular DF e IDF
    df_dict = {t: sum(1 for doc in documentos if t in doc) for t in terminos}
    idf_dict = {t: math.log10(total_docs / df_dict[t]) if df_dict[t] else 0 for t in terminos}
    
    # Calcular TF-IDF
    tfidf_data = []
    for t in terminos:
        fila = []
        for doc in documentos:
            tf = doc.count(t) / len(doc) if len(doc) > 0 else 0
            tfidf = round(tf * idf_dict[t], 3)
            fila.append(tfidf)
        tfidf_data.append(fila)
    
    cols = [f'doc{i+1}' for i in range(total_docs)]
    df = pd.DataFrame(tfidf_data, index=terminos, columns=cols).rename_axis("Término")
 
    
    return df


In [41]:
df = matriz_tfidf('c:/Users/wil_s/Documentos/GitHub/2025A/ir25a/data/01_corpus_turismo.txt')
pd.set_option('display.max_columns', None)
pd.set_option("display.width", None)
pd.set_option('display.max_rows', None)
print(df)


                 doc1   doc2   doc3   doc4   doc5   doc6   doc7   doc8   doc9  \
Término                                                                         
2000            0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000   
a               0.000  0.000  0.000  0.000  0.000  0.000  0.046  0.038  0.000   
agua            0.000  0.000  0.000  0.075  0.000  0.000  0.000  0.000  0.000   
alrededor       0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000   
altitud         0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000   
amazonia        0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000   
andes           0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.075  0.000   
arquitectura    0.000  0.000  0.000  0.000  0.000  0.000  0.093  0.000  0.000   
artesanal       0.000  0.000  0.000  0.000  0.000  0.100  0.000  0.000  0.000   
ascensiones     0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.075  0.000   
atrae           0.000  0.000